In [1]:
import pandas as pd

In [2]:
def getTop(df, column, n=10):
    l = df[column].to_dict().items()
    return sorted(l, key=lambda x : x[-1], reverse=True)[:n]

def getRank(df, column):
        return {k:i for i, (k, v) in enumerate(getTop(df, column, n=None), 1)}

In [65]:
data = pd.read_csv('../data/all_features_unification.csv')
data = data.set_index('uid')

data['positive_text'] = data['positive_text'].fillna(value=0.0)
data[['is_nsfw', 'has_smile', 'number_of_faces', 'has_face']] = \
data[['is_nsfw', 'has_smile', 'number_of_faces', 'has_face']].fillna(0.0).astype(int)

del data['Unnamed: 0']
data.to_csv('../data/all_features_unification.csv', index_label='vk_id')

In [4]:
rating_columns = data.columns[:5].to_list()

ratings = {}
for c in rating_columns:
    ratings[c] = getRank(data, c)
    
ratings = pd.DataFrame(ratings)

In [5]:
ratings['url'] = ratings.index.map(lambda x : f'https://vk.com/id{x}')

In [6]:
resulting_df = pd.concat([ratings, data[['is_nsfw', 'has_smile', 'number_of_faces', 'has_face']]], axis=1)

In [7]:
resulting_df.columns = resulting_df.columns[:-4].to_list() + ['Is NSFW', 'Has Smile', 'Number Of Faces', 'Has Face']

In [13]:
resulting_df

,Friendship Pagerank,Likes Pagerank,Eigencentrality score,Hits in score,Hits out score,url,Is NSFW,Has Smile,Number Of Faces,Has Face
161,12044,4408,7767,6255,6255,https://vk.com/id161,0,0,0,0
169,727,392,7902,5481,5481,https://vk.com/id169,1,0,1,1
204,41208,35736,23474,21901,21901,https://vk.com/id204,0,1,1,1
247,49569,27695,27923,22714,22714,https://vk.com/id247,0,0,0,0
276,30983,30795,24784,24090,24090,https://vk.com/id276,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...
542020665,11368,39867,42875,39217,39217,https://vk.com/id542020665,0,0,0,0
542026027,49340,33431,47835,47571,47571,https://vk.com/id542026027,0,0,2,1
542621368,42131,48150,27374,25455,25455,https://vk.com/id542621368,0,0,0,0
543139718,40328,47499,47556,48406,48406,https://vk.com/id543139718,0,0,0,0


In [35]:
top_1000_friendship_pagerank = set(resulting_df.sort_values('Friendship Pagerank').index.to_list()[:10000])
top_1000_likes_pagerank = set(resulting_df.sort_values('Likes Pagerank').index.to_list()[:10000])
top_1000_eigen = set(resulting_df.sort_values('Eigencentrality score').index.to_list()[:10000])
top_100_hitsin = set(resulting_df.sort_values('Hits in score').index.to_list()[:100])
top_100_hitsout = set(resulting_df.sort_values('Hits out score').index.to_list()[:100])
top_1000_hitsin = set(resulting_df.sort_values('Hits in score').index.to_list()[:10000])
top_1000_hitsout = set(resulting_df.sort_values('Hits out score').index.to_list()[:10000])

In [36]:
len(top_1000_friendship_pagerank.intersection(top_1000_likes_pagerank).intersection(top_1000_eigen))

1776

In [38]:
resulting_df.loc[top_1000_friendship_pagerank.intersection(top_1000_likes_pagerank).intersection(top_1000_eigen)].\
sort_values('Friendship Pagerank').to_csv('./leadership.metrics-top-from-10K-all-scores.csv', index_label='vk_id')

In [40]:
resulting_df['Average Pagerank Rate'] = \
((resulting_df['Friendship Pagerank'].astype(float) + resulting_df['Likes Pagerank'].astype(float))/2).\
apply(lambda x : round(x, 2))
resulting_df['Average Eigencentrality+Hits Rate'] = \
((resulting_df['Eigencentrality score'].astype(float) + resulting_df['Hits in score'].astype(float) + \
  resulting_df['Hits out score'].astype(float))/3).apply(lambda x : round(x, 2))

In [44]:
top_500_pagerank = set(resulting_df.sort_values('Average Pagerank Rate').index.to_list()[:500])
top_500_eigenhits = set(resulting_df.sort_values('Average Eigencentrality+Hits Rate').index.to_list()[:500])

top_1000_pagerank = set(resulting_df.sort_values('Average Pagerank Rate').index.to_list()[:1000])
top_1000_eigenhits = set(resulting_df.sort_values('Average Eigencentrality+Hits Rate').index.to_list()[:1000])

In [43]:
len(top_500_eigenhits.intersection(top_500_pagerank))

33

In [49]:
resulting_df.loc[top_500_eigenhits.intersection(top_500_pagerank)].sort_values('Average Pagerank Rate').\
to_csv('./leadership.metrics-top-avg-500.csv', index_label='vk_id')

In [45]:
len(top_1000_eigenhits.intersection(top_1000_pagerank))

58

In [50]:
resulting_df.loc[top_1000_eigenhits.intersection(top_1000_pagerank)].sort_values('Average Pagerank Rate').\
to_csv('./leadership.metrics-top-avg-1000.csv', index_label='vk_id')

In [60]:
resulting_df['Average by all features'] = resulting_df[resulting_df.columns[:5]].astype(int).sum(axis=1)/5

In [63]:
resulting_df.sort_values('Average by all features').head(500).\
to_csv('./leadership.metrics-top-all-average-500.csv', index_label='vk_id')

In [29]:
resulting_df.loc[top_100_hitsin.intersection(top_100_hitsout)].\
sort_values('Hits in score').to_csv('./leadership.metrics-top-hits.csv', index_label='vk_id')

In [64]:
resulting_df.to_csv('./leadership.metrics-top.csv', index_label='vk_id')